### Laboratorio redes neuronales


In [1]:
"""
Universidad del Valle de Guatemala
Inteligencia Artificial
Catedrático: Samuel Chavez
Pablo Viana - 16091
"""

#Importamos librerias que vamos a utilizar
import numpy as np
import sys
from functools import reduce
from sklearn.model_selection import train_test_split
import time
from scipy import optimize as op
sys.path.insert(1, 'fashion-mnist/utils')

#importamos mnist_reader
import mnist_reader


In [2]:
# ---- Funciones extraídas o basadas a las presentadas en clases virtuales ----
flatten_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

#funcion sigmoide para cada a
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def feed_forward(thetas, X):
    # Lista de activaciones
    a = [X] 
    for i in range(len(thetas)): 
        a.append(
            # a
            sigmoid(
                # z
                np.matmul(
                    # agrega bias
                    np.hstack((
                        np.ones(len(a[i])).reshape(len(a[i]), 1),
                        a[i]
                    )),
                    thetas[i].T
                )
            )
        )
    return a

In [3]:
#funciones para "apachar" e "inflar" matrices
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)
    
    for i in range(layers - 1):
        steps[i+1] = steps[i] + sizes[i]
    
    return [
        flat_thetas[steps[i]: steps[i+1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

def flatmat(list_):
    flat = []
    for matrix in list_:
        flat = [*flat,*(matrix.flatten())]
    return np.array(flat).flatten()

In [4]:
#Función de costo
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1-Y) * np.log(1-a[-1])).sum() / len(X)

In [5]:
#Back propagation
def back_prop(flat_thetas, shapes, X, Y):
    deltas = []
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    a = feed_forward(thetas, X)
    
    deltas = [*range(layers - 1), a[-1] - Y]
    for i in range(layers - 2, 0, -1):
        deltas[i] = np.matmul(deltas[i+1], (thetas[i])[:,1:]) * (a[i] * (1 - a[i]))

    D = []
    for i in range(layers - 1):
        D.append((np.matmul(
        deltas[i+1].T,
        np.hstack((
                    np.ones(len(a[i])).reshape(len(a[i]), 1),
                    a[i]
                ))
        )) /m)

    return flatmat(D)


In [6]:
#Leemos los datos de entenamiento y test
X_train, y_train = mnist_reader.load_mnist('fashion-mnist/data/fashion',kind='train')
X_test, y_test = mnist_reader.load_mnist('fashion-mnist/data/fashion', kind='t10k')

"""
#Dividimos el dataset para obtener una particion de validación
ind1 = np.random.choice(range(X_train.shape[0]),50000,replace=False)
ind2 = list(set(range(X_train.shape[0])) - set(ind1))

X_trainew = X_train[ind1, :]
y_trainew = y_train[ind1].reshape(X_trainew.shape[0],1)
X_val = X_train[ind2, :]
y_val = y_train[ind2].reshape(y_test.shape[0],1)
"""
#Normalizamos los datos de entrenamiento
X = X_train / 1000
m,n = X.shape

#Ajustamos el shape para que tenga la forma (10,1)
Y= np.zeros((X.shape[0], 10))
for i in range(m):
    Y[i][y_train[i]] = 1

Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

#### Hacemos corrida de red neuronal

In [7]:
#Especificamos capas de entrada, escondidas, y de salida para la red neuronal
HIDDEN_NEURONS = 120
OUTPUT_NEURONS = 10

"""
Si la red neuronal tiene Sj unidades en la capa j y S(j+1) en la cada j + 1
=> la shape de la matriz de transicion thetaj es S(j+1) x Sj + 1
""" 
theta_shapes = np.array([
    [HIDDEN_NEURONS, n + 1],
    [OUTPUT_NEURONS, HIDDEN_NEURONS + 1]
])

#Proceso de volver poblar la matriz y volverla un array "aplanar" 
random_thetas = flatten_arrays([
    np.random.rand(*theta_shapes) / 1000
    for theta_shapes in theta_shapes
])


#### Optimización de los thetas por medio de función minimize

In [8]:
print("EMPIEZA OPTIMIZACION DE THETAS")

inicio = time.time()
thetas_optimize = op.minimize(
    fun = cost_function,
    x0 = random_thetas,
    args = (theta_shapes, X, Y),
    method = 'L-BFGS-B',
    jac = back_prop,
    options = {'disp': True, 'maxiter': 100}
)
final = time.time()
t_total = final - inicio

print("OPTIMIZACION TERMINADA tiempo de ejecucion:{}".format(t_total))


EMPIEZA OPTIMIZACION DE THETAS
OPTIMIZACION TERMINADA tiempo de ejecucion:287.33113169670105


In [11]:
#Para asegurar estos thetas generados tras el proceso de optimizacion los guardamos
np.save('top',thetas_optimize.x)

In [10]:
#thetas = np.load(top)
thetas = np.load('top.npy')
thetasop = inflate_matrixes(thetas, theta_shapes)

X = X_test/1000.0
m, n = X.shape
Y = np.zeros((X.shape[0], 10))
for i in range(m):
    Y[i][y_test[i]] = 1

print("PREDICIENDO")
inicio2 = time.time()
a = feed_forward(thetasop, X)
final2 = time.time()
tiempotot = final2 - inicio2
print("TERMINADO en {}".format(tiempotot))

resultados_correctos = np.argmax(a[-1], axis = 1)
buenas = 0
for i in range(len(a[-1])):
    if (resultados_correctos[i] == y_test[i]):
        buenas += 1
    
malas = len(resultados_correctos) - buenas
        
        
print("Buenas: ", buenas)
print("Malas: ", malas)
print("Porcentaje de efectividad del modelo: ",buenas * 100/len(a[-1]))

PREDICIENDO
TERMINADO en 0.14760732650756836
Buenas:  8240
Malas:  1760
Porcentaje de efectividad del modelo:  82.4
